# BlockEncoding class 201

Welcome back! In the previous tutorial, we learned how to "embed" a non-unitary matrix $A$ in the top left block a larger unitary $U$ using the ``BlockEncoding`` class. But just having $A$ block encoded usually isn't enough. Often, we want to compute functions of that matrix—like $e^{-iAt}$ for simulating physics (Hamiltonians), or $A^{-1}$ for solving linear systems, or even applying step-functions or the gaussian as a filter for ground-state preparation. 

Classically, if you have a number $x$, you can calculate $f(x)$ using a Taylor series or a polynomial approximation. In quantum computing, we can actually do the same thing by using an approach called Quantum Signal Processing (QSP). This will be covered towards the end of this tutorial. To get there, we first need to learn how to use the BlockEncoding class to transform into a "Quantum Walk" via Qubitization.

After learning about Qubitization, this tutorial will also explain how to block-encode Chebyshev polynomials and why they are useful. We will then show how to run the algorithms stemming from QSP in Qrisp, and how the ``BlockEncoding`` class makes polynomial transformations, solving linear systems, and hamiltonian simulation as simple as calling ``.poly``, ``.inv``, and ``.sim`` methods respectively.

But first thing's first. Let's break down the concept called qubitization.

## Qubitization
If a Block Encoding is a "static snapshot" of a matrix $A$, Qubitization is what makes it "move". Technically, Qubitization is a method to transform an $(\alpha, m, \epsilon)$-block-encoding of a matrix $A$ into a special unitary operator $W$, often refferod to as the "walk operator". This operator has a nice property: it maps the eigenvalues $\lambda$ of $A$ to the eigenvalues $e^{\pm i \arccos(\lambda/\alpha)}$ in a set of two-dimensional invariant subspaces. 

Given a Hermitian matrix $H$ and its block-encoding $(U, G)$, where $G\ket{0} = \ket{G}$, we use the definition of the reflection operator $R$ acting on the ancilla space as $R = (2\ket{G}\bra{G}_a \otimes \mathbb{1}_a)\otimes \mathbb{1}_{s}$ from [Lemma 1 in Exact and efficient Lanczos method on a quantum computer](https://arxiv.org/pdf/2208.00567). To "qubitize" the encoding, we interleave the SELECT operator with this reflection. The Qubitized Walk Operator $W$ is defined as $W = \text{SELECT}\cdot R$.

Rigorous analysis (see Lin Lin, Chapter 8) shows that if $\ket{\psi_\lambda}$ is an eigenvector of $H/\alpha$ with eigenvalue $\lambda \in [-1, 1]$, the operator $W$ acts on a 2D subspace spanned by $\ket{G}\ket{\psi_\lambda}$ and its orthogonal complement $\ket{\perp}$. Within this subspace, the eigenvalues of $W$ are $\mu_\pm = \lambda \pm i\sqrt{1-\lambda^2} = e^{\pm i \arccos(\lambda)}$. Essentially, Qubitization "lifts" the eigenvalues of our matrix onto the unit circle in the complex plane, allowing us to manipulate them using phase rotations.

This seems like a mouthful, but in order for this tutorial to cater to both developers coming from the classical domain, as well as researchers in quantum computing, it's the "necessary evil". To make it up to you, we're going to show how in Qrisp, you don't need to build these reflections manually. But first, some visual aid so that you see that it's not as complex as it sounds.

![Alt text](walk_operator.png)

In the previous tutorial you've already learned how we use the $\text{SELECT}$ by just calling ``q_switch``. Well, what if we told you that performing the reflection operation above you can just use the ``reflection`` function? Yup, that's the cool thing about modular software development approach Qrisp is taking with its focus on high-level abstractions. Let's get to coding!

### Qubitization in Qrisp as ``.qubitization``

While understanding the internal mechanics of q_switch and reflection is valuable for intuition, Qrisp abstracts this complexity away for standard operations. The BlockEncoding class features a dedicated method, .qubitization(), which automatically constructs the walk operator $W$ from your input matrix.This method handles the heavy lifting: it identifies the necessary reflection operators $R$ and interleaves them with the signal oracle (the block-encoding unitary $U$). If the original block-encoding unitary $U$ isn't Hermitian (i.e., $U^2 \neq \mathbb{1}$), Qrisp automatically handles the Hermitian embedding—often requiring one additional ancilla qubit—to ensure the walk operator remains unitary.Here is how you can transform a Hamiltonian into its qubitized walk operator in just a few lines:

In [9]:
from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Y, Z

# 1. Define a Hamiltonian
# We create a simple Hamiltonian H = X_0 Y_1 + 0.5 Z_0 X_1
H = X(0)*Y(1) + 0.5*Z(0)*X(1)

# 2. Create the initial Block Encoding
# This generates the (U, G) pair discussed above
BE = BlockEncoding.from_operator(H)

# 3. Generate the Qubitized Walk Operator
# This automates the construction of W = SELECT * R
BE_walk = BE.qubitization()

The resulting object, ``BE_walk``, is a new ``BlockEncoding`` instance representing the walk operator $W$. A thing to remember from this example is the fact that when you invoke methods of the ``BlockEncoding`` class like ``.qubitization``, or later ``.poly`` and ``.sim``, Qrisp qubitizes your BlockEncoding object under the hood, handling the ancilla management and reflection logic for you, abstracting away the need to know how to implement these methods as (trigger warning) circuits.

Crucially, the ``.qubitization`` operator is also how oneencodes the Chebyshev polynomials of the Hamiltonian, as we'll learn in the next part of the tutorial. 

### Block encoding Chebyshev polynomials
One of the most powerful features of Qubitization is its natural relationship with Chebyshev polynomials of the first kind, $T_k(x)$, defined as $T_k(\cos \theta) = \cos(k\theta)$. If we apply the walk operator $W$ $k$-times, the resulting unitary $W^k$ contains $T_k(\frac{A}{\alpha})$ block encoded in the top-left block:
$$(\bra{G} \otimes \mathbb{1}) W^k (\ket{G} \otimes \mathbb{1}) = T_k(\frac{A}{\alpha}).$$

"But what's so special about Chebyshev polynomials", you might be wondering. As noted in Lin Lin’s lecture notes, Chebyshev polynomials are "optimal" in two senses:

- Iterative Efficiency: Because $W$ is a single unitary, applying $W^k$ requires only $k$ queries to the block encoding. This is much cheaper than the $O(2^n)$ terms often required by naive Taylor series expansions.

- Approximation Theory: According to the Chebyshev Equioscillation Theorem, $T_k(x)$ provides the best uniform approximation to a function over the interval $[-1, 1]$. This ensures that our quantum algorithm achieves the desired precision $\epsilon$ with the minimum possible quantum resources.

we think that, again, some visual aid is needed. By appyling the ``.qubitization`` operator $k$ times, we block encode the $k$-th Chebyshev polynomial of the first kind $T_k$. If you apply $W^k=(RU)^k$ $k$ times, you get $T_k$ block encoded. Do it once, $k=1$, you get the top left figure. Do it twice ($k=2$), you block-encode $T_2$ (top right figure). Do it $k=5$ times... yup, you guessed it (bottom right figure):

![Alt text](chebyshev.png)

Just as with the basic walk operator, Qrisp abstracts the iterative application of $W$ into a simple method call. The ``BlockEncoding`` class provides a ``.chebyshev(k)`` method, which returns a new block encoding for the $k$-th Chebyshev polynomial $T_k$. This handles the construction of $W^k$ (or the appropriate sequence of reflections and select/``qswitch`` operators) internally.

Here is how to generate and apply a Chebyshev polynomial transformation to a Hamiltonian:

In [10]:
from qrisp import *
from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Y, Z

# 1. Define the Hamiltonian
H = X(0)*X(1) + 0.5*Z(0)*Z(1)

# 2. Create the initial Block Encoding
BE = BlockEncoding.from_operator(H)

# 3. Create the Block Encoding for T_2(H)
# This generates the circuit for the 2nd order Chebyshev polynomial
BE_cheb_2 = BE.chebyshev(k=2)

# 4. Use the new Block Encoding
# For example, applying it via Repeat-Until-Success (RUS) to a state
def operand_prep():
    # Prepare an initial state, e.g., uniform superposition on 2 qubits
    qv = QuantumFloat(2)
    h(qv)
    return qv

@terminal_sampling
def main(BE):
    # Apply the block encoded operator T_2(H) to the state
    qv = BE.apply_rus(operand_prep)()
    return qv

# Execute
result = main(BE_cheb_2)
print(result)

Simulating 1 qubits.. |                                                      | [  0%]

{0.0: 0.4900000153295707, 3.0: 0.48999992592259706, 1.0: 0.010000029373916136, 2.0: 0.010000029373916136}


The ``.chebyshev(k)`` method is particularly useful for building polynomial approximations where $T_k$ terms are the basis functions. By default (``rescale=True``), it returns a block-encoding of $T_k(H)$, managing the normalization factors via Quantum Eigenvalue Transformation (QET) logic. If you need the raw polynomial $T_k(H/\alpha)$ relative to the block-encoding's normalization $\alpha$, you can set ``rescale=False``.

As learned in the previous tutorial you can also perform resource analysis by just calling the ``.resources`` method.

In [12]:
qv = operand_prep()

cheb_resources = BE_cheb_2.resources(qv)
print(cheb_resources)

{'gate counts': {'cx': 12, 'rx': 2, 'gphase': 4, 'p': 2, 'cz': 6, 'u3': 4, 'x': 8}, 'depth': 33}


Let's, at this point, also show how we could do a quick benchmark of the scaling of resources needed. Since the walk operatore $W=(RU)$ is exactly the block encoding overhead for Chebyshev polynomials, we can see how the resources scale with repeated applications of them to block-encode $T_k$.

In [14]:
for k in range(1, 8):
    qv = operand_prep()
    # Generate the k-th Chebyshev Block Encoding
    # We use rescale=False to look at the raw complexity of the walk operator iterations
    BE_cheb = BE.chebyshev(k, rescale=False)
    
    # Extract resource dictionary
    cheb_resources = BE_cheb.resources(qv)
    print(f"k = {k}: {cheb_resources}")

k = 1: {'gate counts': {'cx': 4, 'gphase': 3, 'cz': 2, 'u3': 2, 'z': 1, 'x': 5}, 'depth': 16}
k = 2: {'gate counts': {'cx': 8, 'gphase': 6, 'cz': 4, 'u3': 4, 'z': 2, 'x': 10}, 'depth': 32}
k = 3: {'gate counts': {'cx': 12, 'gphase': 9, 'cz': 6, 'u3': 6, 'z': 3, 'x': 15}, 'depth': 48}
k = 4: {'gate counts': {'cx': 16, 'gphase': 12, 'cz': 8, 'u3': 8, 'z': 4, 'x': 20}, 'depth': 64}
k = 5: {'gate counts': {'cx': 20, 'gphase': 15, 'cz': 10, 'u3': 10, 'z': 5, 'x': 25}, 'depth': 80}
k = 6: {'gate counts': {'cx': 24, 'gphase': 18, 'cz': 12, 'u3': 12, 'z': 6, 'x': 30}, 'depth': 96}
k = 7: {'gate counts': {'cx': 28, 'gphase': 21, 'cz': 14, 'u3': 14, 'z': 7, 'x': 35}, 'depth': 112}


If you look at the printed output of your benchmark, you’ll notice a very satisfying trend: the gate counts and depth grow linearly with $k$.

In the classical world, high-order polynomial approximations often come with a heavy computational tax. On a quantum computer, thanks to Qubitization, the $k$-th Chebyshev polynomial $T_k$ is implemented simply by repeating the walk operator $W$ exactly $k$ times. This efficiency is the "secret sauce" behind many modern quantum algorithms—we get high-precision approximations without the exponential gate-count explosion.

### Quantum Lanczos method

The ability to efficiently block-encode Chebyshev polynomials isn't just a mathematical flex; it’s a prerequisite for one of the most exciting algorithms in recent years: the Quantum Lanczos Method.

As detailed in the paper [Exact and efficient Lanczos method on a quantum computer](https://arxiv.org/pdf/2208.00567), these polynomials are used to construct what is known as a Krylov subspace. By applying different orders of $T_k$ to an initial state, we can "scan" the spectrum of a Hamiltonian.

To put it in a more digestible way, the Lanczos method projects the Hamiltonian into a much smaller, manageable subspace. We then calculate the overlaps (matrix elements) between these Chebyshev-transformed states.

If you're wondering why you should care, this allows for highly accurate Ground State Preparation! By diagonalizing the matrix in this small Krylov subspace, we can find the lowest eigenvalue (the ground state energy) and the corresponding state with far fewer resources than traditional Phase Estimation. 

In Qrisp, the ``.chebyshev`` method serves as the engine for these advanced spectral methods. If you're looking to dive deep into the implementation of this, you can check out the ``lanczos_method`` reference page in our documentation. To provide a glimpse, here is one short example and a comparison to the classical solution to verify the result:

In [ ]:
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()
#### LANCZOS EXAMPLE HERE by using lanczos_alg()

While Chebyshev polynomials are the "optimal" choice for many tasks, they are still just one type of polynomial. What if you want to implement a step function to filter states? Or an inverse function $1/x$ for solving linear systems of equations? Or a complex exponential $e^{-ixt}$ for Hamiltonian simulation?

To do that, we need a more generalized framework that treats the walk operator not just as a repeating block, but as a tunable sequence. This brings us to the "Grand Unified Theory" of quantum algorithms: Quantum Signal Processing (QSP).

In the next chapter, we’ll see how Qrisp takes everything we've learned about block encodings and qubitization to let you implement an arbitrary polynomial transformation by simply calling ``.poly()``.

### Childs Kothari Somma for solving linear systems

## Quantum Signal Processing (QSP)

While LCU and Qubitization allows us to block-encode operators and Chebyshev polynomials $T_k$;the latter by simply repeating a walk operator $W=RU$, Quantum Signal Processing (QSP) provides a way to implement arbitrary polynomial transformations $P(A)$.

At its core, QSP manipulates a single-qubit "signal" using a sequence of rotations. If we have a signal operator $W(x)$ that encodes some value $x \in [-1, 1]$, and we interleave it with a series of phase shifts $e^{i\phi_j Z}$, the resulting product of unitaries can be written as:
$$U_\Phi(x) = e^{i\phi_0 Z} \prod_{j=1}^d W(x) e^{i\phi_j Z}$$

Through a clever choice of the phase angles $\{\phi_0, \phi_1, \dots, \phi_d\}$, the block-encoding (top left block) of this unitary becomes a polynomial $P(x)$. 

The Fundamental Theorem of QSP states that there exists a set of phase angles $\{\phi_0, \dots, \phi_d\}$ such that the top-left block of $U_\Phi$ corresponds to a polynomial $P(A/\alpha)$ where:

- $\text{deg}(P) \leq d$

- $P$ has parity $d \pmod 2$ (it is either purely even or purely odd), and

- $|P(x)| \leq 1$ for all $x \in [-1, 1]$

I know, I know, this was quite a lot of theory, but as usually, we're here to make things simple with Qrisp. We have made it possible for you to not even worry worry about the classical math of finding these angles! Obtaining them involves some heavy Laurent series and optimization already included in Qrisp as an an internal "angle solver" that handles this "classical nightmare" for you. You can therefore treat these complex mathematical transformations as simple method calls!

As a final point of emphasis here, the main advantage of QSP lies in its optimality: it can approximate any continuous function to within error $\epsilon$ using a circuit depth that scales nearly linearly with the complexity of the function, meeting the theoretical lower bounds for quantum query complexity.

### Quantum Eigenvalue and Singular Value Transformation

Building on our discussion of Qubitization and LCU, we can now dive into the "Grand Unification" of quantum algorithms: Quantum Singular Value Transformation (QSVT). In the context of Lin Lin’s lecture notes, these methods represent the most efficient way to process matrices on a quantum computer by treating a matrix as a ``BlockEncoding``.

QSVT applies a polynomial $P$ to the singular values of a matrix $A$ without needing to perform a full Singular Value Decomposition (SVD). 

Disclaimer, a bit more maths before showing examples of how to perform this simply and intuitevely as methods of the class we've been covering.

Consider a matrix $A \in \mathbb{C}^{m \times n}$ with $\|A\| \leq 1$ with SVD of $A = \sum_{i} \sigma_i \ket{w_i} \bra{v_i}$. If we have a $(\alpha, m, \epsilon)$-block encoding $U$, our goal is to construct a new unitary $U_\Phi$ that implements:
$$P(A) = \sum_{i} P(\sigma_i) \ket{w_i} \bra{v_i}.$$

QSVT does this by using Projector-Controlled Phase gates interleaved with the block encoding $U$. Let $\Pi = \ket{0}\bra{0}^a \otimes \mathbb{1}$ be the projector onto the subspace where $A$ lives. The QSVT circuit schematics for a degree-$d$ polynomial is:
$$U_\Phi = e^{i\phi_1(2\Pi - I)} U e^{i\phi_2(2\Pi - I)} U^\dagger e^{i\phi_3(2\Pi - I)} U \dots.$$
To visualize this, the following circuit schematics can help.

![Alt text](BE_QSVT.png)

As promised above, you don't even need to care about these angles with our crispy clean implementation. Generalizing QSP is the final piece of this mosaic.

### Lifting constraints and generalizing QSP
While standard QSVT is a milestone, it is limited by the parity constraint. In simpler terms, that the polynomials must be strictly even or odd, and their coefficients be real (I'm sure that was a social media at some point, right?). 

Recent advancements (e.g., Sünderhauf et al., 2023) have introduced Generalized versions that remove these restrictions.

- GQET (Generalized Quantum Eigenvalue Transformation): Specifically for Hermitian matrices, GQET applies complex polynomials $P(x)$ to eigenvalues. Unlike standard QET, $P(x)$ can have indefinite parity (e.g., $P(x) = x^2 + x + 1$), enabled by using general $SU(2)$ rotations in the signal processing stage.

- GQSVT (Generalized Quantum Singular Value Transformation): This extends QSVT to arbitrary matrices using the generalized framework, allowing for mixed parity polynomials.

Why does this generalization matter so much? It allows for mixed parity polynomial, resulting in you being to implement functions like $e^{-iAt}$ directly without splitting them into sine (odd) and cosine (even) components.

Apart from that, finding phase factors for standard QSVT is often a hard optimization problem scaling as $\tilde{O}(d^2)$. In the Generalized (GQSP) framework, phases can often be computed in linear time $\tilde{O}(d)$, making it significantly more practical for more quantum resource heavy application.

Ok, enough of this, let's now show how you can use, run, simulate, and provide resource analysis for there kinds of algorithms.

## QSP with Qrisp

### Polynomial transformations: ``.poly``

In Qrisp, the BlockEncoding class provides the .poly() method, which leverages GQET to apply a transformation $p(A)$ to a Hermitian matrix. You simply provide the coefficients, and Qrisp’s internal "autopilot" handles the phase solving and circuit construction.Example: Applying a custom polynomialThis example applies $p(A) = 1 + 2A + A^2$ to a matrix $A$ and applies the result to a vector $\ket{b}$.

In [ ]:
import numpy as np
from qrisp import *
from qrisp.block_encodings import BlockEncoding

# Define a Hermitian matrix A and a vector b
A = np.array([[0.73255474, 0.14516978, -0.14510851, -0.0391581],
              [0.14516978, 0.68701415, -0.04929867, -0.00999921],
              [-0.14510851, -0.04929867, 0.76587818, -0.03420339],
              [-0.0391581, -0.00999921, -0.03420339, 0.58862043]])
b = np.array([0, 1, 1, 1])

# Generate a block-encoding and apply the polynomial [1, 2, 1]
BA = BlockEncoding.from_array(A)
BA_poly = BA.poly(np.array([1., 2., 1.]))

# Prepare the state |b>
def prep_b():
    operand = QuantumVariable(2)
    prepare(operand, b)
    return operand

@terminal_sampling
def main():
    # Use Repeat-Until-Success (RUS) to apply the non-unitary polynomial
    operand = BA_poly.apply_rus(prep_b)()
    return operand

res_dict = main()
amps = np.sqrt([res_dict.get(i, 0) for i in range(len(b))])

# Classical verification
c = (np.eye(4) + 2 * A + A @ A) @ b
c = c / np.linalg.norm(c)
print("QUANTUM SIMULATION\n", amps, "\nCLASSICAL SOLUTION\n", c)

### Solving linear systems: ``.inv``

Matrix inversion is implemented via Quantum Eigenvalue Transformation (QET) using a polynomial approximation of $1/x$ over the domain $D_{\kappa} = [-1, -1/\kappa] \cup [1/\kappa, 1]$. The polynomial degree scales as $\mathcal{O}(\kappa \log(\kappa/\epsilon))$, where $\kappa$ is the condition number.

Let's, for example solve the Quantum Linear System Problem $Ax=b$

In [ ]:
# Assuming A and b are defined as above
kappa = np.linalg.cond(A)
BA = BlockEncoding.from_array(A)

# Approximate A^-1 with target precision 0.01 and condition number bound 2
BA_inv = BA.inv(eps=0.01, kappa=2)

@terminal_sampling
def main():
    operand = BA_inv.apply_rus(prep_b)()
    return operand

res_dict = main()
amps = np.sqrt([res_dict.get(i, 0) for i in range(len(b))])

# Classical verification
c = (np.linalg.inv(A) @ b) / np.linalg.norm(np.linalg.inv(A) @ b)
print("QUANTUM SIMULATION\n", amps, "\nCLASSICAL SOLUTION\n", c)

### Hamiltonian simulation: ``.sim``

For a block-encoded Hamiltonian $H$, the .sim() method approximates the unitary evolution $e^{-itH}$. This utilizes the Jacobi-Anger expansion into Bessel functions:$$e^{-it\cos(\theta)} \approx \sum_{n=-N}^{N}(-i)^nJ_n(t)e^{in\theta}$$The truncation error decreases super-exponentially with the order $N$.

We can now simply simulate an Ising Hamiltonian:

In [ ]:
from qrisp.operators import X, Z

def create_ising_hamiltonian(L, J, B):
    return sum(-J * Z(i) * Z(i + 1) for i in range(L-1)) + sum(B * X(i) for i in range(L))

L = 4
H = create_ising_hamiltonian(L, 0.25, 0.5)
BE = BlockEncoding.from_operator(H)

@terminal_sampling
def main(t):
    # Evolve state |0> for time t using order N=8
    BE_sim = BE.sim(t=t, N=8)
    operand = BE_sim.apply_rus(lambda: QuantumFloat(L))()
    return operand

res_dict = main(0.5)

## Conclusion

As hopefully you're now convinced, having block encodings as programming abstractions lowers the barrier to entry for classical developers to dive into the field of state-of-the-art quantum computing algorithms, while at the same time allowing researchers to focus on the algorithm (the function to be applied) rather than the software implementations!

A quick collection of take-home messages after this throrough (and math heavy) tutorial. We have transitioned from basic matrix representations to complex functional analysis on a quantum computer:

- Qubitization is the engine that enables walking through 2D subspaces.

- QSP/GQSP is the steering wheel, allowing you to transform $A$ into almost any $f(A)$.

- Qrisp is the autopilot, solving for phase angles and automating resource management and uncomputation.

If you're eager for more, there is still one tutorial that continues following the concepts, where we take these concepts a step further by introducing Generalized Quantum Signal Processing (GQSP) as a powerful tool for ground state preparation.

In the next tutorial you will explore Ground State Preparation for a 10-qubit antiferromagnetic Heisenberg model. The tutorial covers:

- Gaussian Filtering: How to define a Gaussian filter centered at the minimum eigenvalue to "clean" an initial state.

- Chebyshev Approximation: The process of decomposing this filter into Chebyshev polynomials (you now know everything about), which are natively compatible with quantum block-encodings.

- The RUS Protocol: Using the Repeat-Until-Success (RUS) mechanism in Qrisp to probabilistically project the system into the desired filtered state.

- Performance Verification: A step-by-step comparison showing how the expectation value of the energy significantly drops—indicating a much higher overlap with the true ground state—after the GQSP filter is applied.


### Hamiltonian simulation with LCU in Qrisp: ``.sim``

## Conclusion

  

We’ve moved from building matrices to performing functional analysis on a quantum computer. Here is your cheat sheet:Qubitization is the engine. It turns a Block Encoding into a "Quantum Walk" that stays within tiny 2D subspaces.Chebyshev Polynomials are the language. They are the natural output of repeating a Qubitized operator and are mathematically optimal for approximations.QSP is the steering wheel. By interleaving phase shifts, we can transform $A$ into almost any $f(A)$.Qrisp is the autopilot. It solves for the phase angles and manages the ancillas so you can treat $f(A)$ like a standard programming object.